In [1]:
import os
import cv2
import pandas as pd
from pathlib import Path

# Define the path to the images and the annotation file
images_folder_path = 'Resized_images'
annotations_file_path = 'Resized_ObjectDetection.xlsx'
output_folder_path = 'Annotated images'

# Create the output folder if it doesn't exist
Path(output_folder_path).mkdir(parents=True, exist_ok=True)

# Read the annotations Excel file
annotations = pd.read_excel(annotations_file_path)

# Define colors for bounding boxes (up to 9 features)
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255),
    (255, 255, 0), (255, 0, 255), (0, 255, 255),
    (128, 0, 0), (0, 128, 0), (0, 0, 128)
]

# Function to adjust text size based on image dimensions
def adjust_text_size(image, text, scale_factor=0.5):
    # Get the size of the text box
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale_factor, 1)[0]
    return scale_factor, text_size

# Process each image
for image_name in annotations['fname'].unique():
    # Read the image
    image_path = os.path.join(images_folder_path, f"{image_name}.png")
    image = cv2.imread(image_path)

    # Filter annotations for the current image
    image_annotations = annotations[annotations['fname'] == image_name]

    # Initialize a list to hold the labels and their colors
    labels_and_colors = []

    # Draw each bounding box
    for index, row in image_annotations.iterrows():
        # Get the coordinates and label
        x_min, y_min, x_max, y_max = row['w_min'], row['h_min'], row['w_max'], row['h_max']
        label = row['structure']

        # Choose color
        color = colors[index % len(colors)]

        # Draw the bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

        # Add the label and color to the list
        labels_and_colors.append((label, color))

    # Sort the labels to ensure consistency across images
    labels_and_colors = sorted(set(labels_and_colors), key=lambda x: x[0])

    # Draw the labels with their corresponding colors in the corner of the image
    label_start_y = 10
    for label, color in labels_and_colors:
        scale, text_size = adjust_text_size(image, label)
        cv2.putText(image, label, (10, label_start_y), cv2.FONT_HERSHEY_SIMPLEX, scale, color, 1)
        label_start_y += int(text_size[1] * 1.5)

    # Save the image with bounding boxes in the output folder
    output_image_path = os.path.join(output_folder_path, f"{image_name}_annotated.png")
    cv2.imwrite(output_image_path, image)

print("Annotation process completed.")


Annotation process completed.


In [2]:
import pandas as pd
import os
import cv2

# Constants
ANNOTATION_FILE = 'Resized_ObjectDetection.xlsx'  # Update the path to your Excel file
IMAGE_DIR = 'Resized_images'  # Update the path to your image directory
OUTPUT_DIR = 'checkAnnotated_images'  # Directory to save annotated images
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Load annotations
annotations = pd.read_excel(ANNOTATION_FILE)
annotations = annotations[annotations['structure'].isin(['NT', 'nasal bone'])]

# Function to draw bounding box
def draw_bounding_box(image, bbox, color, structure):
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
    cv2.putText(image, structure, (x_min, y_min-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Process each file
for fname in annotations['fname'].unique():
    image_path = os.path.join(IMAGE_DIR, f"{fname}.png")
    image = cv2.imread(image_path)
    
    image_annotations = annotations[annotations['fname'] == fname]

    for structure in ['NT', 'nasal bone']:
        structure_annotations = image_annotations[image_annotations['structure'] == structure]
        if not structure_annotations.empty:
            h_min, w_min, h_max, w_max = structure_annotations.iloc[0][['h_min', 'w_min', 'h_max', 'w_max']]
            bbox = [w_min, h_min, w_max, h_max]
            color = (0, 255, 0) if structure == 'NT' else (0, 0, 255)  # Green for NT, Red for nasal bone
            draw_bounding_box(image, bbox, color, structure)

    output_path = os.path.join(OUTPUT_DIR, f"{fname}_annotated.png")
    cv2.imwrite(output_path, image)
